# Grid Search

Grid Search works by dividing the search domain into a grid and evaluating each grid vertex. One of its main strengths is that it gives you control over how precise you need the solution to be (e.g. to the nearest integer) but this can lead to computational infeasibility if the precision is set too finely. Alternatively you can specify a number of iterations and let GS work out the appropriate precision to use. It may be worth modifying the code for some problems if, for example, hyp[0] needs to be searched more finely than hype[1].

In [1]:
import itertools
import random as r
import numpy as np

## Objective Functions

In this notebook we'll test the semi-empirical mass formula. It's a formula from physics which gives us the binding energy of a nucleus given its proton number, Z, and neutron number, N. Since physics is effectively trying to maximise the binding energy per nucleon, we have a fun optimization problem if we divide the energy predicted by the SEMF by Z + N. You can play around with physics and see which isotopes would be stable if you changed, say, the Coulomb term! 

SEMF is also useful since it has a known best result of Nickel-62: Z = 28, N = 34. The closer the output is to this, the better our algorithm is doing. Another very stable isotope is Iron-56: Z = 26, N = 30, so if we get a result close to this the algorithm has done well, too.

More about the SEMF here: https://en.wikipedia.org/wiki/Semi-empirical_mass_formula

If you want a different objective function, check out Objectives.ipynb.

In [2]:
def SEMF(
    hypothesis, 
    volumeConstant = 15.8,
    surfaceConstant = 17.8,
    coulombConstant = 0.711,
    asymmetryConstant = 23.7,
    pairingConstant = 11.18
):
    
    #Physical info from hypothesis
    Z = int(hypothesis[0]) #Proton number
    N = int(hypothesis[1]) #Neutron number
    A = Z + N              #Nucleon number
    
    #Calculate each term
    volumeTerm = volumeConstant * A
    surfaceTerm = -surfaceConstant * A ** (2/3)
    coulombTerm = -coulombConstant * Z * (Z - 1) * A ** (-1/3)
    asymmetryTerm = -asymmetryConstant * ((N - Z) ** 2) / A
    
    #Pairing term
    if A % 2 == 1:
        pairingTerm = 0
    elif Z % 2 == 0 and N % 2 == 0:
        pairingTerm = pairingConstant * A ** (-1/2)
    elif Z % 2 == 1 and N % 2 == 1:
        pairingTerm = -pairingConstant * A ** (-1/2)
    else:
        pairingTerm = 0
    output = (volumeTerm + surfaceTerm + coulombTerm + asymmetryTerm + pairingTerm) / A
    return(output)

## Grid Search

In [6]:
class GridSearch:
    def __init__(
        self,
        ranges,
        precision = 1
    ):
        self.ranges = ranges
        self.precision = precision
        
    def generateGrid(self):
        gridEdges = []
        for x in self.ranges:
            gridEdges.append(np.arange(x[0], x[1], self.precision))
        output = list(itertools.product(*gridEdges))
        return(output)
    
    def optimize(self, objectiveFunction):
        bestX = None
        bestY = -float("inf")
        grid = self.generateGrid()
        for newX in grid:
            newY = objectiveFunction(newX)
            if newY > bestY:
                bestX = newX
                bestY = newY
        return(bestX)
gs = GridSearch([[1, 120], [1, 180]])
print(gs.optimize(SEMF))

(np.int64(26), np.int64(32))
